In [ ]:
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")

In [ ]:
#pragma cling load("libapi_eigen_example_rest_client")

In [ ]:
// Let us create a Client object first. This starts by importing the library
#include <apieigen/rest/EigenClient.hpp>

In [ ]:
// Let us instantiate our client
auto client = ansys::rest::client::EigenClient("http:/cpp-rest_server_1:18080");

In [ ]:
// Now, let us check if the connection to the server is adequate by requesting a greeting
client.request_greeting();

In [ ]:
// Once the connection has been checked, let us start declaring some vectors

#include <vector>

std::vector<double> vec1 = {2.0, 3.0, 4.0, 5.0};
std::vector<double> vec2 = {3.0, 7.0, 2.0, 1.0};

In [ ]:
// Let us add the vectors!
client.add_vectors(vec1, vec2)

In [ ]:
// Let us compute the dot product of the vectors!
client.multiply_vectors(vec1, vec2)

In [ ]:
// Now, let us do the same for matrices!
std::vector<std::vector<double>> mat1 = {{2.0, 3.0}, {4.0, 5.0}};
std::vector<std::vector<double>> mat2 = {{3.0, 7.0}, {2.0, 1.0}};

In [ ]:
// Let us add the matrices!
client.add_matrices(mat1, mat2)

In [ ]:
// Let us multiply the matrices!
client.multiply_matrices(mat1, mat2)